## Imports

In [ ]:
import numpy as np 
import torch 
from torch.utils.data import Dataset, DataLoader

## Global varibles

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATASET = 'small16'
BATCH_SIZE = 32
MODEL = 'logreg-20230512T101119Z'

## Load data

In [ ]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X, 
        self.y = y
    
    def __getitem__(self, idx):
        X = self.X[0][idx].astype('float32') 
        y = self.y[idx].astype('float32')
        return X, y
    
    def __len__(self):
        return len(self.y)

# load data
X = np.load(f'./data/{DATASET}/X_train.npy')
y = np.load(f'./data/{DATASET}/y_train.npy')

# make dataset
dataset = Dataset(X, y)

# make dataloader
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


## Load model